In [1]:
##用于可视化
%matplotlib inline  

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#之前reshape默认为True
mnist = input_data.read_data_sets('/home/liuzixuan/机器学习/深度/小练习/MNIST_data',one_hot=True,reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#二维卷积时输入是4阶张量，一维卷积输入是3阶张量
#1代表通道数
mnist.train.images.shape

(55000, 28, 28, 1)

In [6]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None,28,28,1],dtype=tf.float32)
    labels = tf.placeholder(shape=[None,10],dtype=tf.float32)
    
    #卷积层C1 6个5*5的卷积核
    C1_weight = tf.get_variable('c1_w',[5,5,1,6],dtype=tf.float32)
    C1_bias = tf.get_variable('c1_b',[6],dtype=tf.float32)
    
    C1 = tf.nn.conv2d(inputs,C1_weight,[1,1,1,1],'VALID')
    C1 = tf.nn.relu(C1 + C1_bias)#shape=[24,24,6]
    
    #池化层pool1
    pool1 = tf.nn.max_pool(C1,[1,2,2,1],[1,2,2,1],'SAME')#shape=[12,12,6]
    
    #卷基层C2 16个5*5的卷积核
    C2_weight = tf.get_variable('c2_w',[5,5,6,16],dtype=tf.float32)
    C2_bias = tf.get_variable('c2_b',[16],dtype=tf.float32)
    
    C2 = tf.nn.conv2d(pool1,C2_weight,[1,1,1,1],'VALID')
    C2 = tf.nn.relu(C2 + C2_bias)#shape=[8,8,16]
    
    #池化层pool2
    pool2 = tf.nn.max_pool(C2,[1,2,2,1],[1,2,2,1],'SAME')#shape=[4,4,16]
    
    #卷基层C3
    #C3_weight = tf.get_variable('c3_w',[4,4,16,120],dtype=tf.float32)
    #C3_bias = tf.get_variable('c3_b',[120],dtype=tf.float32)
    
    #C3 = tf.nn.conv2d(pool2,C3_weight,[1,1,1,1],'VALID')
    #C3 = tf.nn.relu(C3 + C3_bias)#shape=[1,1,120]
    
    tmp = tf.reshape(pool2,[-1,4 * 4 * 16])
    #tmp = tf.reshape(pool2,[-1,pool2_shape[1]*pool2_shape[2]*pool2_shape[3]])
    #输出层为全连接层
    D1 = tf.keras.layers.Dense(120,activation=tf.nn.relu6)(tmp)
    D2 = tf.keras.layers.Dense(84,activation=tf.nn.relu6)(D1)
    logits = tf.keras.layers.Dense(10,activation=tf.nn.softmax)(D2)
    
    output = tf.nn.softmax(logits)
    
    #loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(labels,output))
    loss = tf.reduce_mean(-tf.reduce_sum(labels * tf.log(output + 1e-17),axis=1))
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output,axis=1),tf.argmax(labels,axis=1)),tf.float32))
    
    ##优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    ##最小化代价函数
    train_op = optim.minimize(loss)
    
    

In [7]:
##训练
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(20000):
        batch_images,batch_labels = mnist.train.next_batch(32)
        res_loss,_ = sess.run([loss,train_op],feed_dict={inputs:batch_images,labels:batch_labels})
        if i%500==0:
            accs = []
            for j in range(10000//32):
                batch_images,batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc,feed_dict={inputs:batch_images,labels:batch_labels})
                accs.append(res_acc)
            m_acc = np.mean(accs)
            print('loss %2.4f,acc %2.2f%%' % (res_loss,m_acc * 100))
            

loss 2.3023,acc 10.59%
loss 2.2998,acc 11.92%
loss 2.2901,acc 10.31%
loss 2.2234,acc 38.02%
loss 1.7415,acc 71.01%
loss 1.6717,acc 79.83%
loss 1.7221,acc 82.64%
loss 1.7252,acc 83.15%
loss 1.7122,acc 84.06%
loss 1.5289,acc 84.71%
loss 1.5434,acc 84.73%
loss 1.5965,acc 85.59%
loss 1.6713,acc 85.76%
loss 1.6393,acc 86.24%
loss 1.5628,acc 86.09%
loss 1.5989,acc 86.35%
loss 1.5230,acc 86.52%
loss 1.6160,acc 86.61%
loss 1.6193,acc 86.94%
loss 1.5503,acc 86.33%
loss 1.5650,acc 87.05%
loss 1.5654,acc 86.95%
loss 1.6812,acc 87.21%
loss 1.5486,acc 86.72%
loss 1.5239,acc 87.01%
loss 1.5600,acc 87.55%
loss 1.6206,acc 87.41%
loss 1.6205,acc 87.59%
loss 1.7196,acc 87.71%
loss 1.5774,acc 87.42%
loss 1.5652,acc 87.41%
loss 1.5864,acc 87.61%
loss 1.5780,acc 87.58%
loss 1.6454,acc 87.44%
loss 1.5224,acc 87.90%
loss 1.5855,acc 87.94%
loss 1.6154,acc 87.70%
loss 1.6493,acc 87.50%
loss 1.6652,acc 88.00%
loss 1.4977,acc 87.91%
